##Convert Annotation txt to geojson 

Bounding Box Coordinate
<br/>Class Bounding Box Coordinate

#Bounding Box Coordinate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
!pip install geojson

In [4]:
import os
import shutil
import pprint
from glob import glob
from termcolor import colored

In [5]:
import re
import math
import geojson
import rasterio
from osgeo import gdal
import numpy as np

In [6]:
def make_bbox_paths(folder_name, file_extension):
  
  bbox_paths = glob(folder_name + file_extension)
  bbox_paths = sorted(bbox_paths, reverse=False)
  bbox_paths = sorted(bbox_paths, key=len)

  return bbox_paths

In [7]:
def make_image_paths(folder_name, bbox_paths):

  image_paths = []
  for bbox_path in bbox_paths:
    img_path = bbox_path.replace('txt', 'tif')
    image_path = folder_name + img_path.split('/')[-1]
    image_paths.append(image_path)

  return image_paths

In [8]:
def make_geojson_paths(folder_name, image_paths):

  geojson_paths = []
  for image_path in image_paths:
    geo_path = image_path.replace('tif', 'geojson')
    geojson_path = folder_name + geo_path.split('/')[-1]
    geojson_paths.append(geojson_path)

  return geojson_paths

In [9]:
def init_geo_dictionary():

  geo_dictionary = {
  "type": "FeatureCollection",
  "name": "fair1m",
  "crs": { "type": "name", "properties": { "name": "" } },
  "features": [
  # lcg
  # { "type": "Feature", "properties": { "id": 1 }, "geometry": { "type": "MultiPolygon",
  #                                                             "coordinates": [ [ ] ] } }
  ]
  }
  
  return geo_dictionary

In [10]:
def init_bbox_coordinate(bbox_path):

  result = []

  with open(bbox_path , 'r') as f:
    while True:
      line = f.readline()
      if not line: break
      a = re.findall(r"[-+]?\d*\.\d+|\d+", line)   
      result.append([float(a[i]) for i in range(len(a)) if i != (len(a) - 1)])
    f.close()

  return result

In [11]:
def digit_length(n):
    return int(math.log10(n)) + 1 if n else 0

In [12]:
def make_geo_dictionary(image_path, result, geo_dictionary):
  
  ds = gdal.Open(image_path)
  metadata = gdal.Info(ds, format='json')

  pixel_size = metadata['size']
  cornerCoordinates = metadata['cornerCoordinates']
  pixel_h = metadata['size'][0]
  pixel_w = metadata['size'][1]

  lowerLeft = cornerCoordinates['lowerLeft']
  lowerRight = cornerCoordinates['lowerRight']
  upperLeft = cornerCoordinates['upperLeft']
  upperRight = cornerCoordinates['upperRight']

  latitude = lowerLeft[0]
  latitude2 = lowerRight[0]
  longtitude = upperLeft[1]
  longtitude2 = lowerLeft[1]
    
  wgs_h = abs(latitude - latitude2)
  wgs_w = abs(longtitude - longtitude2)

  epsg = re.findall(r'\d+', metadata['coordinateSystem']['wkt'])

  if digit_length(abs(latitude)) >= 5:
    geo_dictionary['crs']['properties']['name'] = 'EPSG:' + epsg[-1]
  
  elif digit_length(abs(latitude)) < 5:
     geo_dictionary['crs']['properties']['name'] = "urn:ogc:def:crs:OGC:1.3:CRS84"

  point_count = 0         # lcg
  for each_bbox in result:
    x1 = each_bbox[0]; y1 = each_bbox[1]
    x2 = each_bbox[2]; y2 = each_bbox[3]
    x3 = each_bbox[4]; y3 = each_bbox[5]
    x4 = each_bbox[6]; y4 = each_bbox[7]

    transform_x1 = latitude + ((x1 / pixel_w) * wgs_w)
    transform_y1 = longtitude - ((y1 / pixel_h) * wgs_h)
    transform_x2 = latitude + ((x2 / pixel_w) * wgs_w)
    transform_y2 = longtitude - ((y2 / pixel_h) * wgs_h)    
    transform_x3 = latitude + ((x3 / pixel_w) * wgs_w)
    transform_y3 = longtitude - ((y3 / pixel_h) * wgs_h)
    transform_x4 = latitude + ((x4 / pixel_w) * wgs_w)
    transform_y4 = longtitude - ((y4 / pixel_h) * wgs_h)

    # point_count = 0         # lcg
    '''
    for key, value in geo_dictionary.items():
      point_count += 1
      if point_count == len(geo_dictionary.items()):
        for i, j in value[0].items():
          if i == 'geometry':
            bbox_coordinate = [[transform_x1, transform_y1], [transform_x2, transform_y2],
                                [transform_x3, transform_y3], [transform_x4, transform_y4]]
            j['coordinates'][0].append(bbox_coordinate)
    '''
    # lcg
    bbox_coordinate = [[[transform_x1, transform_y1], [transform_x2, transform_y2],
                [transform_x3, transform_y3], [transform_x4, transform_y4], [transform_x1, transform_y1]]]
    geo_dictionary['features'].append({"type": "Feature", "properties": { "id": point_count }, 
        "geometry": { "type": "MultiPolygon", "coordinates": [bbox_coordinate] }})
    point_count += 1    

  return geo_dictionary

In [13]:
def make_geojson_dictionary(bbox_path, image_path):

  geo_dictionary = init_geo_dictionary()
  result = init_bbox_coordinate(bbox_path)
  geojson_dictionary = make_geo_dictionary(image_path, result, geo_dictionary)
  
  return geojson_dictionary

In [14]:
def save_geojson_dictionary(bbox_paths, image_paths, json_paths):

  for bbox_path, image_path, json_path in zip(bbox_paths, image_paths, json_paths):
    geojson_dictionary = make_geojson_dictionary(bbox_path, image_path)
    with open(json_path, 'w') as f:
      geojson.dump(geojson_dictionary, f)    

In [17]:
# lcg
#folder_train_json = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson/'
#os.makedirs(folder_train_json)

train_bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/train_super/', '*.txt')
train_image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_train/', train_bbox_paths)
train_geojson_paths = make_geojson_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson/', train_image_paths)
#train_geojson_paths = make_geojson_paths(folder_train_json, train_image_paths)

In [ ]:
# lcg
folder_val_json = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/val_geojson/'
os.makedirs(folder_val_json)

val_bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/val_super/', '*.txt')
val_image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_val/', val_bbox_paths)
# val_geojson_paths = make_geojson_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson/val_geojson/', val_image_paths)
val_geojson_paths = make_geojson_paths(folder_val_json, val_image_paths)

In [ ]:
save_geojson_dictionary(train_bbox_paths, train_image_paths, train_geojson_paths)
save_geojson_dictionary(val_bbox_paths, val_image_paths, val_geojson_paths)

In [44]:
geojson_path = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/val_geojson/6.geojson'

with open(geojson_path) as f:
    geojson_file = geojson.load(f)

geojson_file_name = geojson_path.split("/")[-1]   
print(colored(geojson_file_name, attrs=['bold']))
print()
pprint.pprint(geojson_file, width=160)

6.geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[139.715187, 35.56254],
                                              [139.715133, 35.56254],
                                              [139.715133, 35.562559],
                                              [139.715187, 35.562559],
                                              [139.715187, 35.56254]]]],
                            'type': 'MultiPolygon'},
               'properties': {'id': 0},
               'type': 'Feature'},
              {'geometry': {'coordinates': [[[[139.715181, 35.56257],
                                              [139.715125, 35.56257],
                                              [139.715125, 35.562586],
                                              [139.715181, 35.562586],
                                              [139.715181, 35.56257]]]],
                            'type': 'MultiPolygon'},
               

#Class Bounding Box Coordinate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
!pip install geojson

In [19]:
import os
import shutil
import pprint
from glob import glob

In [20]:
import re
import math
import geojson
import rasterio
from osgeo import gdal
import numpy as np

In [21]:
def make_bbox_paths(folder_name, file_extension):
  
  bbox_paths = glob(folder_name + file_extension)
  bbox_paths = sorted(bbox_paths, reverse=False)
  bbox_paths = sorted(bbox_paths, key=len)

  return bbox_paths

In [22]:
def make_image_paths(folder_name, bbox_paths):

  image_paths = []
  for bbox_path in bbox_paths:
    img_path = bbox_path.replace('txt', 'tif')
    image_path = folder_name + img_path.split('/')[-1]
    image_paths.append(image_path)

  return image_paths

In [23]:
def make_class_geojson_paths(folder_name, image_paths):

  ship_geojson_paths = []
  vehicle_geojson_paths = []
  airplane_geojson_paths = []

  for image_path in image_paths:
    ship_bbox_geo = image_path.split('/')[-1].replace('.tif', '_ship.geojson')
    ship_bbox_geojson = folder_name + ship_bbox_geo
    ship_geojson_paths.append(ship_bbox_geojson)

    vehicle_bbox_geo = image_path.split('/')[-1].replace('.tif', '_vehicle.geojson')
    vehicle_bbox_geojson = folder_name + vehicle_bbox_geo
    vehicle_geojson_paths.append(vehicle_bbox_geojson)

    airplane_bbox_geo = image_path.split('/')[-1].replace('.tif', '_airplane.geojson')
    airplane_bbox_geojson = folder_name + airplane_bbox_geo
    airplane_geojson_paths.append(airplane_bbox_geojson)
    
  return ship_geojson_paths, vehicle_geojson_paths, airplane_geojson_paths

In [24]:
def init_geo_dictionary():

  geo_dictionary = {
  "type": "FeatureCollection",
  "name": "fair1m",
  "crs": { "type": "name", "properties": { "name": "" } },
  "features": [
  # lcg
  # { "type": "Feature", "properties": { "id": 1 }, "geometry": { "type": "MultiPolygon",
  #                                                             "coordinates": [ [ ] ] } }
  ]
  }
  
  return geo_dictionary

In [25]:
def init_class_bbox_coordinate(bbox_path):

  ship_list = []
  vehicle_list = []
  airplane_list = []

  with open(bbox_path , 'r') as f:
    while True:
      line = f.readline()
      if not line: break
      a = re.findall(r"[-+]?\d*\.\d+|\d+", line)
      b = re.findall(r"[A-Za-z_]+", line)[-1]
      if b == 'ship':
        ship_list.append([float(a[i]) for i in range(len(a)) if i != (len(a) - 1)])
      elif b == 'vehicle':
        vehicle_list.append([float(a[i]) for i in range(len(a)) if i != (len(a) - 1)])
      elif b == 'airplane':
        airplane_list.append([float(a[i]) for i in range(len(a)) if i != (len(a) - 1)])
    f.close()

  return ship_list, vehicle_list, airplane_list

In [26]:
def digit_length(n):
    return int(math.log10(n)) + 1 if n else 0

In [27]:
def make_class_geo_dictionary(image_path, result, geo_dictionary):
  
  ds = gdal.Open(image_path)
  metadata = gdal.Info(ds, format='json')

  pixel_size = metadata['size']
  cornerCoordinates = metadata['cornerCoordinates']
  pixel_h = metadata['size'][0]
  pixel_w = metadata['size'][1]

  lowerLeft = cornerCoordinates['lowerLeft']
  lowerRight = cornerCoordinates['lowerRight']
  upperLeft = cornerCoordinates['upperLeft']
  upperRight = cornerCoordinates['upperRight']

  latitude = lowerLeft[0]
  latitude2 = lowerRight[0]
  longtitude = upperLeft[1]
  longtitude2 = lowerLeft[1]
    
  wgs_h = abs(latitude - latitude2)
  wgs_w = abs(longtitude - longtitude2)

  epsg = re.findall(r'\d+', metadata['coordinateSystem']['wkt'])

  if digit_length(abs(latitude)) >= 5:
    geo_dictionary['crs']['properties']['name'] = 'EPSG:' + epsg[-1]
  
  elif digit_length(abs(latitude)) < 5:
     geo_dictionary['crs']['properties']['name'] = "urn:ogc:def:crs:OGC:1.3:CRS84"

  point_count = 0         # lcg
  for each_bbox in result:
    x1 = each_bbox[0]; y1 = each_bbox[1]
    x2 = each_bbox[2]; y2 = each_bbox[3]
    x3 = each_bbox[4]; y3 = each_bbox[5]
    x4 = each_bbox[6]; y4 = each_bbox[7]

    transform_x1 = latitude + ((x1 / pixel_w) * wgs_w)
    transform_y1 = longtitude - ((y1 / pixel_h) * wgs_h)
    transform_x2 = latitude + ((x2 / pixel_w) * wgs_w)
    transform_y2 = longtitude - ((y2 / pixel_h) * wgs_h)    
    transform_x3 = latitude + ((x3 / pixel_w) * wgs_w)
    transform_y3 = longtitude - ((y3 / pixel_h) * wgs_h)
    transform_x4 = latitude + ((x4 / pixel_w) * wgs_w)
    transform_y4 = longtitude - ((y4 / pixel_h) * wgs_h)

      # point_count = 0         # lcg
    '''
    for key, value in geo_dictionary.items():
      point_count += 1
      if point_count == len(geo_dictionary.items()):
        for i, j in value[0].items():
          if i == 'geometry':
            bbox_coordinate = [[transform_x1, transform_y1], [transform_x2, transform_y2],
                                [transform_x3, transform_y3], [transform_x4, transform_y4]]
            j['coordinates'][0].append(bbox_coordinate)
    '''
    # lcg
    bbox_coordinate = [[[transform_x1, transform_y1], [transform_x2, transform_y2],
                [transform_x3, transform_y3], [transform_x4, transform_y4], [transform_x1, transform_y1]]]
    geo_dictionary['features'].append({"type": "Feature", "properties": { "id": point_count }, 
        "geometry": { "type": "MultiPolygon", "coordinates": [bbox_coordinate] }})
    point_count += 1    

  return geo_dictionary

In [28]:
def make_class_geojson_dictionary(bbox_path, image_path):

  ship_list, vehicle_list, airplane_list = init_class_bbox_coordinate(bbox_path)  

  geo_dictionary = init_geo_dictionary()
  ship_geojson_dictionary = make_class_geo_dictionary(image_path, ship_list, geo_dictionary)

  geo_dictionary = init_geo_dictionary()
  vehicle_geojson_dictionary = make_class_geo_dictionary(image_path, vehicle_list, geo_dictionary)

  geo_dictionary = init_geo_dictionary()
  airplane_geojson_dictionary = make_class_geo_dictionary(image_path, airplane_list, geo_dictionary)

  return ship_geojson_dictionary, vehicle_geojson_dictionary, airplane_geojson_dictionary

In [29]:
def save_class_geojson_dictionary(bbox_paths, image_paths, ship_json_paths, vehicle_json_paths, airplane_json_paths):

  for bbox_path, image_path, ship_json_path, vehicle_json_path, airplane_json_path \
  in zip(bbox_paths, image_paths, ship_json_paths, vehicle_json_paths, airplane_json_paths):

    ship_geojson_dictionary, vehicle_geojson_dictionary, airplane_geojson_dictionary \
     = make_class_geojson_dictionary(bbox_path, image_path)

    with open(ship_json_path, 'w') as f:
      geojson.dump(ship_geojson_dictionary, f)   
    
    f.close()

    with open(vehicle_json_path, 'w') as f:
      geojson.dump(vehicle_geojson_dictionary, f)  

    f.close()

    with open(airplane_json_path, 'w') as f:
      geojson.dump(airplane_geojson_dictionary, f)    
    
    f.close()

In [30]:
def make_geo_dictionary(image_path, result, geo_dictionary):
  
  ds = gdal.Open(image_path)
  metadata = gdal.Info(ds, format='json')

  pixel_size = metadata['size']
  cornerCoordinates = metadata['cornerCoordinates']
  pixel_h = metadata['size'][0]
  pixel_w = metadata['size'][1]

  lowerLeft = cornerCoordinates['lowerLeft']
  lowerRight = cornerCoordinates['lowerRight']
  upperLeft = cornerCoordinates['upperLeft']
  upperRight = cornerCoordinates['upperRight']

  latitude = lowerLeft[0]
  latitude2 = lowerRight[0]
  longtitude = upperLeft[1]
  longtitude2 = lowerLeft[1]
    
  wgs_h = abs(latitude - latitude2)
  wgs_w = abs(longtitude - longtitude2)

  epsg = re.findall(r'\d+', metadata['coordinateSystem']['wkt'])

  if digit_length(abs(latitude)) >= 5:
    geo_dictionary['crs']['properties']['name'] = 'EPSG:' + epsg[-1]
  
  elif digit_length(abs(latitude)) < 5:
     geo_dictionary['crs']['properties']['name'] = "urn:ogc:def:crs:OGC:1.3:CRS84"

  point_count = 0         # lcg
  for each_bbox in result:
    x1 = each_bbox[0]; y1 = each_bbox[1]
    x2 = each_bbox[2]; y2 = each_bbox[3]
    x3 = each_bbox[4]; y3 = each_bbox[5]
    x4 = each_bbox[6]; y4 = each_bbox[7]

    transform_x1 = latitude + ((x1 / pixel_w) * wgs_w)
    transform_y1 = longtitude - ((y1 / pixel_h) * wgs_h)
    transform_x2 = latitude + ((x2 / pixel_w) * wgs_w)
    transform_y2 = longtitude - ((y2 / pixel_h) * wgs_h)    
    transform_x3 = latitude + ((x3 / pixel_w) * wgs_w)
    transform_y3 = longtitude - ((y3 / pixel_h) * wgs_h)
    transform_x4 = latitude + ((x4 / pixel_w) * wgs_w)
    transform_y4 = longtitude - ((y4 / pixel_h) * wgs_h)

    # point_count = 0         # lcg
    '''
    for key, value in geo_dictionary.items():
      point_count += 1
      if point_count == len(geo_dictionary.items()):
        for i, j in value[0].items():
          if i == 'geometry':
            bbox_coordinate = [[transform_x1, transform_y1], [transform_x2, transform_y2],
                                [transform_x3, transform_y3], [transform_x4, transform_y4]]
            j['coordinates'][0].append(bbox_coordinate)
    '''
    # lcg
    bbox_coordinate = [[[transform_x1, transform_y1], [transform_x2, transform_y2],
                [transform_x3, transform_y3], [transform_x4, transform_y4], [transform_x1, transform_y1]]]
    geo_dictionary['features'].append({"type": "Feature", "properties": { "id": point_count }, 
        "geometry": { "type": "MultiPolygon", "coordinates": [bbox_coordinate] }})
    point_count += 1    

  return geo_dictionary

In [39]:
train_bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/train_super/', '*.txt')
train_image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_train/', train_bbox_paths)
train_ship_geojson_paths, train_vehicle_geojson_paths, train_airplane_geojson_paths \
 = make_class_geojson_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson_class/', train_image_paths)

In [38]:
val_bbox_paths = make_bbox_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/val_super/', '*.txt')
val_image_paths = make_image_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/images_val/', val_bbox_paths)
val_ship_geojson_paths, val_vehicle_geojson_paths, val_airplane_geojson_paths \
 = make_class_geojson_paths('/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/val_geojson_class/', val_image_paths)

In [ ]:
save_class_geojson_dictionary(train_bbox_paths, train_image_paths, \
                              train_ship_geojson_paths, train_vehicle_geojson_paths, train_airplane_geojson_paths)
save_class_geojson_dictionary(val_bbox_paths, val_image_paths, \
                              val_ship_geojson_paths, val_vehicle_geojson_paths, val_airplane_geojson_paths)

In [43]:
ship_geojson_path = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson_class/6_ship.geojson'
vehicle_geojson_path = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson_class/6_vehicle.geojson'
airplane_geojson_path = '/content/drive/MyDrive/Aiffelthon/mmrotate/data/fair1m2.0/geojson_fixed/train_geojson_class/6_airplane.geojson'

with open(ship_geojson_path) as f:
    ship_geojson_file = geojson.load(f)

with open(vehicle_geojson_path) as f:
    vehicle_geojson_file = geojson.load(f)

with open(airplane_geojson_path) as f:
    airplane_geojson_file = geojson.load(f)

ship_geojson_file_name = ship_geojson_path.split("/")[-1]   
vehicle_geojson_file_name = vehicle_geojson_path.split("/")[-1]   
airplane_geojson_file_name = airplane_geojson_path.split("/")[-1]   

print(colored(ship_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(ship_geojson_file, width=160)

print("\n" * 2)

print(colored(vehicle_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(vehicle_geojson_file, width=160)

print("\n" * 2)

print(colored(airplane_geojson_file_name, attrs=['bold']))
print()
pprint.pprint(airplane_geojson_file, width=160)

6_ship.geojson

{'crs': {'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [[[[139.662889, 35.443785],
                                              [139.662873, 35.443767],
                                              [139.6628, 35.443834],
                                              [139.662814, 35.44385],
                                              [139.662889, 35.443785]]]],
                            'type': 'MultiPolygon'},
               'properties': {'id': 0},
               'type': 'Feature'},
              {'geometry': {'coordinates': [[[[139.662817, 35.443461],
                                              [139.662776, 35.443498],
                                              [139.662795, 35.44352],
                                              [139.662835, 35.443482],
                                              [139.662817, 35.443461]]]],
                            'type': 'MultiPolygon'},
        